In diesem Skript wird nicht auf ein sequentielles Model (Sequential) gesetzt, sondern auf ein Model

In [7]:
import os # Arbeiten mit Pfaden
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPool2D

from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from mnistData import MNIST

In [8]:
np.random.seed(0)
tf.random.set_seed(0)

In [9]:
def build_model() -> Model:
    input_img = Input(shape=(28,28,1))

    x = Conv2D(filters=16, kernel_size=3, padding='same')(input_img)
    x = Activation("relu")(x)
    x = Conv2D(filters=16, kernel_size=3, padding='same')(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)

    x = Conv2D(filters=32, kernel_size=3, padding='same')(input_img)
    x = Activation('relu')(x)
    x = Conv2D(filters=32, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPool2D()(x)

    x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPool2D()(x)

    x = Flatten()(x)
    x = Dense(units=128)(x)
    x = Activation("relu")(x)
    x = Dense(units=10)(x)
    y_pred = Activation("softmax")(x)

    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )

    # In diesem Beispiel muss das Modell schon hier gebuildet werden, damit das Objekt der Wrapper-Klasse übergeben werden kann
    model.compile( 
        loss='categorical_crossentropy',
        optimizer='Adam',
        metrics=['accuracy']
    )
    
    model.summary()

    return model

In [10]:
data = MNIST(with_normalization=True)
x_train, y_train = data.get_train_set()

keras_clf = KerasClassifier(
    build_fn=build_model, # Funktionsobjekt übergeben, Fit-Methode ist hier intern gelöst und Parameter müssen angegeben werden
    epochs=10,
    batch_size=128,
    verbose=1
)

scores = cross_val_score(
    estimator=keras_clf,
    X=x_train,
    y=y_train,
    cv=3, # 3 Folds
    n_jobs=1
)

C:\Users\gutsc\AppData\Local\Temp/ipykernel_35316/33715981.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  keras_clf = KerasClassifier(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_14 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 activation_14 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 conv2d_15 (Conv2D)          (None, 28, 28, 32)        9248      
                                                                 
 activation_15 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                             

In [11]:
print(f'Score list: {scores}')
mean_scores=np.mean(scores)
std_scores=np.std(scores)
print(f'Mean Acc: {mean_scores:.6} (+/- {std_scores:.6})')

Score list: [0.99084997 0.98879999 0.98694998]
Mean Acc: 0.988867 (+/- 0.00159286)
